# 필요 함수 import

In [1]:
import requests
from bs4 import BeautifulSoup

import re
from collections import Counter
import nltk
from konlpy.tag import Okt

import pandas as pd
import numpy as np

import pickle

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gyuu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1차 : 쿠팡 상품 리스트 추출
> categoryId
- 밀키트 : 502482
- 1국탕전골 : 502483
- 2덮밥/비빔밥 : 502484
- 3스테이크/고기 502485
- 4면/파스타/감바스 502486
- 5분식 502487
- 6중식요리 502490
- 7기타요리 502491

In [2]:
# 쿠팡 페이지 호출 함수
def get_coupang_item(URL, user_agt):
    headers = {"User-Agent":user_agt
               , "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    res = requests.get(URL, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

## 필요 정보만 추출
- 카테고리 대분류, 상품id, 상품only, 상품명(전체), 정가, 할인율, 판매가, 100g당_가격, 별점, 리뷰수, 품절여부

In [3]:
#카테고리 대분류 라벨링 함수
def category_number(categoryId, category):
    return category.get(categoryId)


#해당 카테고리의 페이지 전체 수 가져오기
def page_number(categoryId, user_agt):
    URL = 'https://www.coupang.com/np/categories/'+categoryId+"?page=1"
    cg_soup_base = get_coupang_item(URL, user_agt)
    page_all = cg_soup_base.select('#product-list-paging > div > a')
    page_list = []
    for page_num in page_all:
        page_num = page_num.text
        try:
            page_num = int(page_num)
            page_list.append(page_num)
        except:
            pass
    return page_list[-1]


#1차 쿠팡 상품 리스트 전처리 함수
def p_list_preprocessing(cg_soup, category_num):
    #상품id, data-item-id, data-vendor-item-id
    bady_pdt = cg_soup.select_one("a.baby-product-link")
    productId = bady_pdt['data-product-id']
    itemsId = bady_pdt['data-item-id']
    vendorItemId = bady_pdt['data-vendor-item-id']

    # 상품명
    name = cg_soup.select_one("div.name").text.replace('\n', '').strip()
    name_only = name.split(', ')[0]

    # 판매가
    price_value = cg_soup.select_one("strong.price-value").text.replace(',','')
    
    try:
        # 정가
        base_price = cg_soup.select_one("del.base-price").text.replace("\n",'').strip().replace(',','')
        # 할인율
        discount_pcg = cg_soup.select_one("span.discount-percentage").text
    except:
        base_price, discount_pcg = price_value, 0

    #100g당 가격
    try:
        unit_price = cg_soup.select_one("span.unit-price").text.replace("\n", '').strip()
    except:
        unit_price = np.NaN
    
    try:
        # 별점
        star = cg_soup.select_one("span.star > em").text
        # 리뷰수
        rating_count = cg_soup.select_one("span.rating-total-count").text
        rating_count = re.sub(r'[^0-9]','',rating_count)
    except:
        star, rating_count = 0, 0

    # 품절여부 > 품절이면 1, 아니면 0
    try:
        out_of_stock = cg_soup.select_one("div.out-of-stock").text.replace("\n", '').strip()
        out_of_stock = 1
    except:
        out_of_stock = 0
    
    #DataFrame으로 병합
    coupang_item = pd.DataFrame({'카테고리명':category_num,'상품id':productId, 'data-item-id':itemsId, 'data-vendor-item-id':vendorItemId
                                , '상품':name_only,'상품명':name, '정가':base_price, '할인율':discount_pcg, '판매가':price_value
                                , '100g당_가격':unit_price, '별점':star, '리뷰수':rating_count, '품절여부':out_of_stock}
                                , index=[0])
    return coupang_item

## 쿠팡 상품 리스트 추출 실행부
> 밀키트 추가 필터링 후 전체 크롤링

- 추가 전처리 해야하는 부분
1. 100g당 가격 결측치 처리
2. 구성정보 텍스트 전처리 : 명사 추출 후 재료만 남기기
3. 구성정보 결측치 채우기 : 텍스트로 구성정보가 존재하는 경우만 >> 그외 결측치 삭제
4. 상품명에서 음식명만 추출 > 세부 카테고리 정리

In [4]:
#categoryId 기반 카테고리 대분류 정의
category = {'502483':1, '502484':2, '502485':3, '502486':4, '502487':5, '502490':6, '502491':7}
category_keys = list(category.keys())

#상품 리스트를 담을 빈 dataframe 생성
item_list = ['카테고리명','상품id','data-item-id','data-vendor-item-id','상품','상품명','정가',
             '할인율','판매가','100g당_가격','별점','리뷰수','품절여부']
coupang_items = pd.DataFrame(columns=item_list)

#쿠팡 상품 리스트 추출
cg_url = 'https://www.coupang.com/np/categories/'
user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

for categoryId in category_keys:
    page_num = page_number(categoryId, user_agt) #해당 카테고리의 페이지 마지막 페이지 번호 추출
    for page in range(1, page_num+1):
        URL = cg_url+categoryId+"?page="+str(page)+'&filter=1%23attr_12406%2419087%40DEFAULT'
        
        #페이지 호출
        cg_soup_base = get_coupang_item(URL, user_agt)
        cg_soup2 = cg_soup_base.select("#productList li")  

        #상품 리스트 추출 및 전처리
        for cg_soup in cg_soup2:
            coupang_item = p_list_preprocessing(cg_soup, category_number(categoryId, category))
            coupang_items = pd.concat([coupang_items,coupang_item], ignore_index=True)

coupang_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                370 non-null    object
 1   상품id                 370 non-null    object
 2   data-item-id         370 non-null    object
 3   data-vendor-item-id  370 non-null    object
 4   상품                   370 non-null    object
 5   상품명                  370 non-null    object
 6   정가                   370 non-null    object
 7   할인율                  370 non-null    object
 8   판매가                  370 non-null    object
 9   100g당_가격             329 non-null    object
 10  별점                   370 non-null    object
 11  리뷰수                  370 non-null    object
 12  품절여부                 370 non-null    object
dtypes: object(13)
memory usage: 37.7+ KB


In [5]:
#전체 중복행 확인
coupang_items.loc[coupang_items.duplicated(subset=['상품id']), :]

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부


In [6]:
#중복행 삭제
coupang_items.drop_duplicates('상품id', inplace=True)
coupang_items.reset_index(drop=True, inplace=True)

# 2차 : 구성 정보 추출
- 상품id, 구성정보

In [7]:
#2차 : 구성정보 추출 함수
def p_c_extraction(productId, p_c_page):
    product_composition = '0'
    for item in p_c_page:
        s = item.text.replace('\n','').strip()
        if product_composition == '2': #3: 재료부분만 추출
            product_composition = s
        elif product_composition =='1':#2: 불필요한 text 제외
            product_composition='2'
        elif '구성 정보' in s :         #1: '구성 정보' 존재여부 확인
            product_composition = '1'

    #4: 없으면 결측값 처리
    if product_composition.isdigit():
        product_composition = np.NaN
    
    data_item = pd.DataFrame({'상품id':productId, '구성정보':product_composition}, index=[0])
    return data_item

## 2차 실행부

In [8]:
#2차 실행부
data_items = pd.DataFrame(columns=['상품id', '구성정보'])

for idx in range(len(coupang_items)):
    #각 상품 상세페이지 호출
    product_url = 'https://www.coupang.com/vp/products/'
    productId = coupang_items.loc[idx, '상품id'] #data-product-id
    itemsId = coupang_items.loc[idx, 'data-item-id']  #data-item-id
    vendorItemId = coupang_items.loc[idx, 'data-vendor-item-id'] #data-vendor-item-id
    URL = product_url+str(productId)+'/items/'+str(itemsId)+'/vendoritems/'+str(vendorItemId)
    user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

    pdt_soup = get_coupang_item(URL, user_agt)
    
    #'구성정보' 추출
    p_c_page = pdt_soup.find('body').children
    data_item = p_c_extraction(productId, p_c_page)
    data_items = pd.concat([data_items, data_item], ignore_index=True)

data_items.head()

,상품id,구성정보
0,4926044090,NaN
1,1866720935,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ..."
2,2202010823,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다."
3,1717552921,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성..."
4,6310053892,"소고기, 콩나물, 대파, 양파, 깻잎, 소스로 구성되어 있습니다."


# 상세페이지 이미지에서 구성정보 추출


In [9]:
def get_coupang_image(img_URL, user_agt):
    headers = {"User-Agent":user_agt
               , "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    page = requests.get(img_URL, headers= headers).text
    #coupang_image_soup = BeautifulSoup(page.text, "html.parser")
    #return coupang_image_soup
    return page

In [10]:
# 구성정보 페이지에서 전체 이미지 링크 추출 -> 리스트화
image_link = []
product_url = 'https://www.coupang.com/vp/products/'
image_productId = list(coupang_items.loc[coupang_items['상품명'].str.contains('곰곰|딜리조이|프렙')]['상품id'].values)
image_dataitemId = list(coupang_items.loc[coupang_items['상품명'].str.contains('곰곰|딜리조이|프렙')]['data-item-id'].values)
image_datavendorId = list(coupang_items.loc[coupang_items['상품명'].str.contains('곰곰|딜리조이|프렙')]['data-vendor-item-id'].values)

for product in range(len(image_productId)):
    img_URL = product_url + str(image_productId[product]) + '/items/' +str(image_dataitemId[product])+'/vendoritems/'+str(image_datavendorId[product])
    user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

    page = get_coupang_image(img_URL, user_agt)

    link_start = []
    for text in re.finditer('//thumbnail',page):
        link_start.append(text.start())
        
    link_end = []
    for text in re.finditer('g"}],',page):
        link_end.append(text.start())
    
    for num in range(len(link_start)):
        image_link.append('https:' + page[int(link_start[num]):int(link_end[num]) + 1])
        

In [11]:
## 상세페이지에서 전체 이미지 저장
import urllib.request

count = 0
for url in image_link:
    count += 1
    path = "C:/coding/image/" + (str(count) + ".jpg")
    urllib.request.urlretrieve(url, path)


In [12]:
# 이미지에서 텍스트 추출 코드 with pytesserect
import pytesseract
import cv2 
import matplotlib.pyplot as plt
from PIL import Image

extracted_text = []
for num in range(1, count + 1):
    file = 'C:/coding/image/' + str(num) + '.jpg'
    image = Image.open(file)
    if image.size[1] >= 4000:
        croppedImage = image.crop((0,2000,image.size[0],3350))
        croppedImage.save('C:/coding/image/cropped' + str(num) + '.jpg')
        file = 'C:/coding/image/cropped' + str(num) + '.jpg'
        image = cv2.imread(file)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        text = pytesseract.image_to_string(rgb_image, lang='kor')
        extracted_text.append(text)
    else:
        image = cv2.imread(file)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        text = pytesseract.image_to_string(rgb_image, lang='kor')
        extracted_text.append(text)

In [13]:
# 결측값 중 곰곰 밀키트 구성정보만 테서렉트로 이미지 텍스트 추출 후,
# 오타 교정 위해 전처리 작업
gomgom_text = [i for i in extracted_text if "구성되어 있어요"  in i] 
gomgom = []
for i in gomgom_text:
    sentence = re.sub('(<([^>]+)>)', '', i)
    sentence = re.sub('[^ ㄱ-ㅎㅏ-ㅣ각-힣]', ' ', sentence)
    gomgom.append(sentence)
    
gom_typo_list = [] 

for gom in gomgom:
    gg = gom.split(' ')
    gom_typo = []
    for i in range(len(gg)):
        if len(gg[i]) > 1:
            gom_typo.append(gg[i])
        else:
            pass
    gom_typo_list.append(gom_typo)

In [14]:
# gom_type_list 피클화
import pickle

with open('gom_typo.pkl','wb') as f:
    pickle.dump(gom_typo_list,f)

# 곰곰 구성재료 오타 교정 (Colab)

In [15]:
# !pip install symspellpy jamo hangul_utils
# !wget https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/ko/ko_50k.txt
# from symspellpy import SymSpell, Verbosity
# from symspellpy_ko import KoSymSpell, Verbosity
# from hangul_utils import split_syllable_char, split_syllables, join_jamos
# from jamo import h2j, j2hcj
# import pandas as pd
# import unicodedata

# def to_jamos(text):
#   return j2hcj(h2j(text))


# 만개의 레시피 재료 목록 자소 분리
# vocab = pd.read_csv('만개의레시피_재료_목록_오타처리용.txt', sep = " ", names = ["term", "count"])
# vocab.head()
# vocab.info()
# vocab = vocab.dropna(axis=0)

# vocab.term = vocab.term.map(to_jamos)
# vocab.to_csv("만개의레시피_재료_목록_오타처리용_decomposed.txt", sep=" ", header=None, index=None)
# vocab.head()

In [16]:
# # 저장했던 리스트 불러옴
# with open('gom_typo.pkl','rb') as f:
#   gom = pickle.load(f)
  
  
# for g in gom:
#     idx = g.index('있어요')
#     del g[0:idx + 1]

# for g in gom:
#   if '일은' in g :
#     idx = g.index('일은')
#     del g[idx :]

#   elif 'ㅇㅇ' in g :
#     idx = g.index('ㅇㅇ')
#     del g[idx :]
  
#   else:
#     pass

# sym_spell = SymSpell(max_dictionary_edit_distance = 3)
# dictionary_path = '만개의레시피_재료_목록_오타처리용_decomposed.txt'
# sym_spell.load_dictionary(dictionary_path, 0, 1)

# # gom_typo_fix에 오타 교정된 재료 목록 저장
# gom_typo_fix = []
# for g in gom:
#   g_typo_fix = []
#   for term in g:
#     term = split_syllables(term) 
#     suggestions = sym_spell.lookup(term, Verbosity.ALL, max_edit_distance = 3)
#     for sugg in suggestions :
#       if sugg.distance < 2:
#         typofix = join_jamos(sugg.term)
#         g_typo_fix.append(typofix)
#         break
#       else:
#         pass
#   gom_typo_fix.append(g_typo_fix)
  
  
# gom_typo_fix를 다시 피클로 저장
# with open('gom_typo_fix.pkl','wb') as f:
#     pickle.dump(gom_typo_fix,f)


## 결측치에 곰곰 채우기

In [17]:
#곰곰 상품id 추출
gomgom_productId = list(coupang_items.loc[coupang_items['상품'].str.contains('곰곰')]['상품id'].values)

#곰곰 이미지에서 추출한 구성정보 list
with open('gom_typo_fix.pkl', 'rb') as fr:
    gomgom_data = pickle.load(fr)

#상품id + 구성정보
gomgom_df = pd.DataFrame(zip(gomgom_productId, gomgom_data), columns=['상품id', '구성정보'])

### 곰곰 결측치 채우기
for i in range(len(gomgom_df)):
    idx = data_items[data_items['상품id'] == gomgom_productId[i]].index.values
    gomgom = gomgom_df.loc[i,'구성정보']
    data_items.loc[idx, '구성정보'] = ', '.join(gomgom)
data_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품id    370 non-null    object
 1   구성정보    329 non-null    object
dtypes: object(2)
memory usage: 5.9+ KB


## 구성정보 전처리 후 재대입

In [18]:
#구성정보 재료 전처리
def p_c_preprocessing(product_composition):
    product_composition = re.sub(r'\\n', '', product_composition)
    product_composition = re.sub(r'\)', ',', product_composition)
    product_composition = re.sub(r'[^ㄱ-ㅣ가-힣a-zA-Z\,\(\:]', ' ', product_composition)
    product_composition = product_composition.strip()
    
    replace_list = ['으로 구성되어 있습니다','으로 구성되어있습니다','로 구성되어 있습니다','로 구성되어 있어요','로 구성되어있습니다']
    for r in replace_list:
        product_composition = product_composition.replace(r, '')
    
    replace_list = ['과', '와']
    for r in replace_list:
        product_composition = product_composition.replace(r, ',') 
    
    # 추가 작업에 따라 추가 여부 결정
    # if ('야채' in product_composition) or ('채소(' in product_composition) or ('채소 :' in product_composition):
    #     replace_list = ['야채 세트', '혼합 채소', '혼합 해물', '채소(', '채소 :']
    #     for r in replace_list:
    #         product_composition = re.sub('[\(\)\:]', ',', product_composition)
    #         product_composition = product_composition.replace(r, '')
    # else:
    product_composition = re.sub('[\(\)\:]', ',', product_composition)

    return product_composition

In [19]:
#요리재료내용 결측치 삭제
data_items_drop = data_items[data_items.loc[:,'구성정보'].isnull()].index
data_items_drop = data_items.drop(data_items_drop)
data_items_drop.reset_index(drop=True, inplace=True)
data_items_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품id    329 non-null    object
 1   구성정보    329 non-null    object
dtypes: object(2)
memory usage: 5.3+ KB


>> 소고기 == 쇠고기 이건 어디에..?

In [20]:
#요리재료_전처리(new column) 
data_items_drop['구성정보_전처리'] = np.NaN

for idx in range(len(data_items_drop)):
    ingredient = data_items_drop.loc[idx, '구성정보']
    ingredients = p_c_preprocessing(ingredient)
    ingredients = ingredients.split(',')
    
    ingre_lists = []
    for i in ingredients:
        i = i.strip()
        replace_list = ['칼국수 면', '다진 마늘', '대구 곤이', '명태 곤이']
        if i in replace_list:
            i = i.split(' ')
            i = ''.join(i)

        i = i.split(' ')
        if i[-1] == '등' :  # 등을 그대로 두는 것이 나을지 지금처럼 빼는게 나을지..?
            i.pop(-1)
            i = ' '.join(i)
            ingre_lists.append(i)
        else:
            i = ' '.join(i)
            ingre_lists.append(i)
        
    #null값 제거
    ingre_lists = list(filter(None, ingre_lists))

    #요리재료_전처리(new column)에 추가
    data_items_drop.loc[idx, '구성정보_전처리'] = ', '.join(ingre_lists)

data_items_drop.iloc[0:10, :]

,상품id,구성정보,구성정보_전처리
0,4926044090,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면..."
1,1866720935,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육, 소곱창, 소대창, 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ..."
2,2202010823,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물, 채소, 연두부, 다시팩, 소스"
3,1717552921,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스"
4,6310053892,"소고기, 콩나물, 대파, 양파, 깻잎, 소스로 구성되어 있습니다.","소고기, 콩나물, 대파, 양파, 깻잎, 소스"
5,1637283332,"순두부, 애호박, 소고기, 팽이버섯, 대파, 소스","순두부, 애호박, 소고기, 팽이버섯, 대파, 소스"
6,188130036,"소고기, 당면, 팽이버섯, 소스, 참기름 등으로 구성되어 있습니다.","소고기, 당면, 팽이버섯, 소스, 참기름"
7,1946869683,"애호박, 양파, 대파, 청양고추, 두부, 팽이버섯","애호박, 양파, 대파, 청양고추, 두부, 팽이버섯"
8,6764028903,"기타 수산물가공품(명태알, 대구곤이, 새우), 채소(무, 대파, 청양고추), 소스로...","기타 수산물가공품, 명태알, 대구곤이, 새우, 채소, 무, 대파, 청양고추, 소스"
9,188130043,"소고기, 소스, 쥬키니호박, 팽이버섯, 대파, 청양고추, 두부로 구성되어 있습니다.","소고기, 소스, 쥬키니호박, 팽이버섯, 대파, 청양고추, 두부"


# 비교를 위한 리스트 생성

In [21]:
# 만개의 레시피 재료 목록 read
ingre_10000 = []
with open('만개의레시피_재료_목록.txt', 'r') as f :
    lines = f.readlines()
    lines = str(lines).split(',')
    for line in lines:
        ingre_10000.append(line) 
len(ingre_10000)

21806

In [22]:
#쿠팡 재료 목록 생성
ingre_coupang_list = data_items_drop['구성정보_전처리'].dropna()
ingre_coupang_lists = ingre_coupang_list.to_list()

ingre_coupangs = []
for item in ingre_coupang_lists:
    ingre_coupang=[]
    items = item.split(', ')
    for i in items:
        ingre_coupang.append(i)
    ingre_coupangs.append(ingre_coupang)

# #쿠팡 재료 목록 저장
# with open('쿠팡_재료_목록.txt', 'w') as f :
#     for ingre in ingre_coupang:
#         f.write(ingre)

In [23]:
#쿠팡에만 있는 재료만
ingre_not_10000 = []
for items in ingre_coupangs :
    not_10000 = []
    for i in items:
        if not i in ingre_10000 and i != '':
            not_10000.append(i)
    ingre_not_10000.append(not_10000)
print(len(ingre_not_10000))
ingre_not_10000

329


[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['기타 수산물가공품'],
 [],
 [],
 [],
 ['바지락맛 액상 소스'],
 ['토자연 우사골육수  팩', '참소스  팩'],
 ['디포리 국물용팩'],
 [],
 [],
 [],
 [],
 ['해산물 팩', '야채팩'],
 [],
 [],
 ['야채 세트', '야채 세트'],
 ['반각 가리비', '야채 세트', '야채 세트'],
 [],
 ['김치찌개 소스'],
 ['백목이버섯'],
 ['야채팩', '청국장  조리 시 물    mL는 추가로 준비해주세요'],
 [],
 [],
 ['닭고기 포장육', '수산물가공품'],
 ['부대찌개 소스'],
 ['혼합 채소'],
 ['야채 세트', '야채 세트'],
 ['야채 세트', '바른 마늘'],
 [],
 ['야채세트', '바른마늘'],
 ['잡채 면', '간장 배 양념'],
 ['정통 순두부찌개순두부', '순두부 찌개양념장'],
 ['소고기 포장육', '소스  종'],
 ['김치 왕만두', '혼합 채소', '혼합 해물', '자숙홍합살'],
 ['해물장 등 짜글이를 만드는데 필요한 재료들'],
 [],
 ['야채 세트', '바른 마늘'],
 ['야채 세트', '야채 세트'],
 ['야채 세트', '건조 매생이'],
 ['고기국수육수  팩', '새우맛양념  팩'],
 ['야채 세트'],
 [],
 [],
 [],
 [],
 [],
 ['주름 미더덕', '얼큰 분말 스프'],
 [],
 ['분말 수프'],
 ['야채 세트'],
 [],
 ['알탕 전용 소스'],
 ['야채 세트', '바른 마늘'],
 [],
 ['기타 수산물가공품'],
 [],
 ['야채 세트'],
 [],
 ['얼큰육수베이스  팩', '새우맛양념  팩'],
 ['기타 수산물가공품', '대왕오징어'],
 ['매운탕 양념'],
 ['다채로운 모양의 어묵들'],
 ['해물장 등 김치찌개를 조리하는데 필요한 재료들'],
 [],
 ['야채 세트', '바른 마늘'],
 ['해물

In [24]:
data_items_drop['구성정보_전처리']

0      쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...
1      양념육, 소곱창, 소대창, 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...
2                                   해물, 채소, 연두부, 다시팩, 소스
3             소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스
4                               소고기, 콩나물, 대파, 양파, 깻잎, 소스
                             ...                        
324                                              오징어, 소스
325    고기 손만두, 소고기, 채소, 청경채, 대파, 청양고추, 홍고추, 버섯, 팽이버섯,...
326    소고기, 야채  종, 적양배추, 깻잎, 건면, 소스  종, 절임식품, 야채  종, ...
327          빵, 양파, 고수, 절임식품, 잔파, 오이, 소스, 마요네즈, 양조간장, 치즈
328    새우, 홍합, 비엔나소시지, 아침 찰옥수수, 야채 세트, 감자, 양파, 마늘, 하얀...
Name: 구성정보_전처리, Length: 329, dtype: object

In [25]:
#일치하는 재료 리스트로 만들고 데이터 프레임으로 수정
ingre_10000s = []
for items in ingre_coupangs :
    ingre_10000_list = []
    for i in items:
        if i in ingre_10000:
            ingre_10000_list.append(i)
    ingre_10000s.append(ingre_10000_list)
print(len(ingre_10000s))
ingre_10000s

329


[['쇠고기',
  '새송이버섯',
  '표고버섯',
  '초간장소스',
  '칠리소스',
  '샤브용육수',
  '생면',
  '칼국수면',
  '숙면',
  '감자수제비',
  '깻잎',
  '팽이버섯',
  '배추',
  '청경채'],
 ['양념육',
  '소곱창',
  '소대창',
  '전골용 육수',
  '새송이버섯',
  '느타리버섯',
  '소고기',
  '두부',
  '배추',
  '양파',
  '깻잎',
  '라면사리'],
 ['해물', '채소', '연두부', '다시팩', '소스'],
 ['소시지', '오뗄팜S', '프레스햄', '소고기', '치즈', '라면사리', '채소', '김치', '소스'],
 ['소고기', '콩나물', '대파', '양파', '깻잎', '소스'],
 ['순두부', '애호박', '소고기', '팽이버섯', '대파', '소스'],
 ['소고기', '당면', '팽이버섯', '소스', '참기름'],
 ['애호박', '양파', '대파', '청양고추', '두부', '팽이버섯'],
 ['명태알', '대구곤이', '새우', '채소', '무', '대파', '청양고추', '소스'],
 ['소고기', '소스', '쥬키니호박', '팽이버섯', '대파', '청양고추', '두부'],
 ['두부', '청국장', '애호박', '돼지고기', '대파', '양파', '혼합장'],
 ['돼지고기', '모듬햄', '대파', '양파', '양배추', '라면사리', '부대찌개소스', '김치', '슬라이스', '치즈'],
 ['바지락', '생합', '모시조개', '파', '고추', '무'],
 ['소 스지', '전각 슬라이스'],
 ['낙지', '백합', '대파', '청양고추', '무'],
 ['알배기', '적근대', '청경채', '느타리', '칼국수', '팽이버섯', '샤브육수', '칠리소스', '유자폰즈'],
 ['미역', '쇠고기', '육수', '볶음', '맛기름'],
 ['두부', '쇠고기', '소스', '팽이버섯', '청양고추', '애호박', '대파', '감

In [26]:
for idx in range(len(ingre_10000s)):
    data_items_drop.loc[idx, '구성정보_전처리'] = ','.join(ingre_10000s[idx])
data_items_drop.head()

,상품id,구성정보,구성정보_전처리
0,4926044090,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기,새송이버섯,표고버섯,초간장소스,칠리소스,샤브용육수,생면,칼국수면,숙면,감자수..."
1,1866720935,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎..."
2,2202010823,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물,채소,연두부,다시팩,소스"
3,1717552921,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지,오뗄팜S,프레스햄,소고기,치즈,라면사리,채소,김치,소스"
4,6310053892,"소고기, 콩나물, 대파, 양파, 깻잎, 소스로 구성되어 있습니다.","소고기,콩나물,대파,양파,깻잎,소스"


## 긴 텍스트에서 사전과 일치하는 요리재료만 추출

In [27]:
#요리재료 사전과 일치여부를 통해 요리재료 추출
# ingredients_list = []
# for coupangs in ingre_coupangs :
#     ingre_list = []
#     for coupang in coupangs:
#         if coupang in ingre_10000:
#             ingre_list.append(coupang)
#         else:
#             ingre = [ man for man in ingre_10000 if man in coupang]
#             ingre_list.extend(ingre)
#     ingredients_list.append(ingre_list)
#     print(ingre_list)
#ingredients_list

# 쿠팡 상품 리스트와 구성정보 병합

In [28]:
#쿠팡 상품 리스트와 구성정보 병합
coupang = pd.merge(coupang_items, data_items_drop)
coupang.to_csv('coupang_filtering.csv')
coupang.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329 entries, 0 to 328
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                329 non-null    object
 1   상품id                 329 non-null    object
 2   data-item-id         329 non-null    object
 3   data-vendor-item-id  329 non-null    object
 4   상품                   329 non-null    object
 5   상품명                  329 non-null    object
 6   정가                   329 non-null    object
 7   할인율                  329 non-null    object
 8   판매가                  329 non-null    object
 9   100g당_가격             290 non-null    object
 10  별점                   329 non-null    object
 11  리뷰수                  329 non-null    object
 12  품절여부                 329 non-null    object
 13  구성정보                 329 non-null    object
 14  구성정보_전처리             329 non-null    object
dtypes: object(15)
memory usage: 41.1+ KB


In [29]:
coupang.head()

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리
0,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.5,15815,0,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기,새송이버섯,표고버섯,초간장소스,칠리소스,샤브용육수,생면,칼국수면,숙면,감자수..."
1,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,9%,23900,"(100g당 1,499원)",4.5,11387,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎..."
2,1,2202010823,3744622966,71729775756,미소프레쉬 6컵 푸짐한 대구탕,"미소프레쉬 6컵 푸짐한 대구탕, 1225g, 1개",14200,0,14200,"(100g당 1,159원)",4.5,12166,0,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물,채소,연두부,다시팩,소스"
3,1,1717552921,2923167957,70911802261,프레시지 더큰 햄가득 부대전골,"프레시지 더큰 햄가득 부대전골, 868g, 1개",14900,0,14900,"(100g당 1,717원)",5.0,36803,0,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지,오뗄팜S,프레스햄,소고기,치즈,라면사리,채소,김치,소스"
4,1,6310053892,13092768757,80353833768,마이셰프 EBS 최요비 산더미 소고기 콩불 2인분,"마이셰프 EBS 최요비 산더미 소고기 콩불 2인분, 810g, 1개",14900,0,14900,"(100g당 1,840원)",4.5,2942,0,"소고기, 콩나물, 대파, 양파, 깻잎, 소스로 구성되어 있습니다.","소고기,콩나물,대파,양파,깻잎,소스"


In [30]:
#구성정보 결측치 확인
print('결측치 개수: ',len(coupang.loc[coupang['구성정보_전처리']=='']))
coupang.loc[coupang['구성정보_전처리']=='']

결측치 개수:  9


,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리
80,1,2140731697,3635578248,71621124717,설래담 민어 매운탕 (냉동),"설래담 민어 매운탕 (냉동), 680g, 1개",15190,0,15190,"(100g당 2,234원)",4.0,339,0,"신선한 민어, 풍성한 야채 믹스, 얼큰한 풍미의 매운탕 소스, 시원한 맛을 더해줄 ...",
141,3,5965059575,10701364629,77982083715,마이셰프 명절 밀키트 버섯가득 소불고기전골 905g + 소고기버섯잡채 571g + ...,마이셰프 명절 밀키트 버섯가득 소불고기전골 905g + 소고기버섯잡채 571g + ...,43700,15%,36900,NaN,4.5,21,0,"버섯가득 소불고기 전골, 궁중 소불고기, 소고기 버섯 잡채로 구성되어 있습니다.",
142,3,5965059546,10701364550,77982083557,마이셰프 명절 밀키트 궁중 소고기 갈비찜 1026g + 소고기버섯잡채 571g + ...,마이셰프 명절 밀키트 궁중 소고기 갈비찜 1026g + 소고기버섯잡채 571g + ...,66700,11%,58900,NaN,5.0,24,0,"궁중 소고기 갈비찜, 일품 소고기 육전, 소고기 버섯 잡채로 구성되어 있습니다.",
247,7,6928382170,16756449794,83937960368,셰프초이스 어제 버무린 매콤한 오징어볶음 씨키트 750g,"셰프초이스 어제 버무린 매콤한 오징어볶음 씨키트 750g, 1세트",14900,0,14900,NaN,3.5,6,0,"매콤한 오징어볶음, 자른 대파, 자른 양배추, 사리 소면으로 구성되어 있습니다.",
250,7,6703653767,15536769131,82755955008,프렙 이탈리안 감바스 알 아히요 (냉동),"프렙 이탈리안 감바스 알 아히요 (냉동), 300g, 1개",13200,0,13200,"(100g당 4,400원)",4.5,48,0,포장 정보,
263,7,6928382821,16756453661,83937964890,셰프초이스 어제 버무린 오리지널 오징어볶음 씨키트 750g,"셰프초이스 어제 버무린 오리지널 오징어볶음 씨키트 750g, 1세트",14900,0,14900,NaN,4.5,2,0,"오리지널 오징어볶음, 자른 대파, 자른 양배추, 사리 소면으로 구성되어 있습니다.",
265,7,6703653639,15536768551,82755954355,프렙 버터 전복 리조또 (냉동),"프렙 버터 전복 리조또 (냉동), 427g, 1개",11600,17%,9600,"(100g당 2,248원)",4.5,60,0,포장 정보,
276,7,6811645649,16116095590,83313001742,집반찬연구소 간편 짜장용 손질야채 240g x 2개 + 청정원 직화 짜장 80g 세트,집반찬연구소 간편 짜장용 손질야채 240g x 2개 + 청정원 직화 짜장 80g 세...,7900,0,7900,NaN,5.0,8,0,청정원 직화짜장과 손질 야채로 구성되어 있습니다.,
314,7,6703653501,15536767789,82755953691,프렙 볼로네제 리조또 (냉동),"프렙 볼로네제 리조또 (냉동), 520g, 1개",11200,0,11200,"(100g당 2,154원)",4.5,44,1,포장 정보,


In [31]:
#우선 결측치 제거
coupang_null_list = coupang.loc[coupang['구성정보_전처리']==''].index
coupang.drop(coupang_null_list, inplace=True)



In [32]:
coupang.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 328
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                320 non-null    object
 1   상품id                 320 non-null    object
 2   data-item-id         320 non-null    object
 3   data-vendor-item-id  320 non-null    object
 4   상품                   320 non-null    object
 5   상품명                  320 non-null    object
 6   정가                   320 non-null    object
 7   할인율                  320 non-null    object
 8   판매가                  320 non-null    object
 9   100g당_가격             286 non-null    object
 10  별점                   320 non-null    object
 11  리뷰수                  320 non-null    object
 12  품절여부                 320 non-null    object
 13  구성정보                 320 non-null    object
 14  구성정보_전처리             320 non-null    object
dtypes: object(15)
memory usage: 40.0+ KB


# 상세카테고리 추가

In [479]:
def category_name(product):
    if ('닭' in product) or ('백숙' in product) : name = '닭'
    elif '구이' in product : name = '구이'
    elif ('전골' in product)or('나베' in product)or('스키야키' in product) or ('샤브샤브' in product): name = '전골'
    elif '족발' in product : name = '족발'
    elif '국수' in product : name = '국수'
    elif ('해물탕' in product) or ('연포탕' in product) or ('해신탕' in product) or ('꽃게탕' in product) or ('해물 누룽지탕' in product) or ('조개탕' in product) or ('바지락탕' in product): name = '해물탕'
    elif ('해물뚝배기' in product) or ('문어오리탕' in product) or ('홍합탕' in product) : name = '해물탕'
    elif ('매운탕' in product) or ('대구탕' in product) or ('동태탕' in product) or ('지리탕' in product)  or ('맑은탕' in product) or ('맑음탕' in product): name = '생선탕'
    elif ('알탕' in product) or ('알이 맛있탕' in product) or ('곤이탕' in product): name = '알탕'

    elif ('찌개' in product) or ('짜글이' in product) or ('짬뽕 순두부' in product): name = '찌개/짜글이'

    elif ('면' in product)or('소바' in product)or('짜장' in product)or('누들' in product)or('짬뽕' in product) : name = '면'
    elif ('우동' in product)or('라면' in product) : name = '면'
    elif '스테이크' in product : name = '스테이크'

    elif ('파스타' in product) or ('라자냐' in product) : name = '파스타'
    


    elif '리조또' in product : name = '리조또' 
        
    elif '마라탕' in product : name = '마라탕'
    elif '마파두부' in product : name = '마파두부'

    elif ('떡볶이' in product) or ('떡볶잉' in product) : name = '떡볶이'
        
    elif '덮밥' in product : name = '덮밥'
    elif '순대' in product : name = '순대'
        
    elif '무침' in product : name = '무침'
    elif '감바스' in product : name = '감바스'
    
    elif ('콩불' in product) or ('불고기' in product) or ('비빔밥' in product) : name='한식'
    elif ('부침개' in product) or ('조림' in product) : name = '한식'
    
    elif ('해물' in product)or('씨푸드' in product)or('씨키트' in product)or('꽃게' in product)or('새우' in product): name = '씨푸드'
    elif ('쉬림프' in product)or('연어' in product)or('조개' in product)or('문어' in product): name = '씨푸드'

    

    elif ('커리' in product) or ('카레' in product) : name = '카레'
    
    elif '샌드위치' in product : name = '샌드위치'
    elif ('볶음' in product) or ('잡채' in product)  : name = '볶음'
    
    elif '쌈' in product : name = '쌈'
    elif ('국' in product) or ('육개장' in product) or ('파개장' in product) or ('수제비' in product) or ('칼제비' in product) : name = '국'
    elif '찜' in product : name = '찜'
    else: name='기타'
        
    return name

In [492]:
coupang['상세카테고리'] = coupang['상품'].apply(lambda x : category_name(x))
coupang.loc[coupang['상세카테고리'] == '볶음']

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리
204,4,6071367429,11210885873,78488424594,이랜드이츠 애슐리 고기듬뿍 잡채,"이랜드이츠 애슐리 고기듬뿍 잡채, 557g, 1개",12800,0,12800,"(100g당 2,298원)",4.5,588,1,"돼지고기, 소스, 참기름, 식용유, 참깨, 야채(피망, 느타리버섯, 양파, 당근),...","돼지고기,소스,참기름,식용유,참깨,야채,피망,느타리버섯,양파,당근,당면",볶음
206,4,293473513,926572584,5302426619,마이셰프 소고기버섯잡채,"마이셰프 소고기버섯잡채, 571g, 1개",14900,0,14900,"(100g당 2,609원)",4.5,1276,1,"건면, 소고기, 새송이버섯, 느타리버섯, 홍파프리카, 청피망, 알가열제품, 참깨, ...","건면,소고기,새송이버섯,느타리버섯,홍파프리카,청피망,참깨,소스,건목이버섯,표고버섯채",볶음
213,6,293473507,926572569,5302426608,마이셰프 매콤 소고기 고추 잡채 꽃빵,"마이셰프 매콤 소고기 고추 잡채 꽃빵, 570g, 1개",13900,21%,10900,"(100g당 1,912원)",5.0,12437,0,"1. 꽃빵, 소고기, 손질한 양파와 파프리카, 죽순, 표고버섯, 피망, 청양고추,...","꽃빵,소고기,파프리카,죽순,표고버섯,피망,청양고추",볶음
217,6,6927813935,16753225569,83934802592,프레시지 99's fresh 고추잡채와 꽃빵,"프레시지 99's fresh 고추잡채와 꽃빵, 661g, 1개",16640,0,16640,"(100g당 2,517원)",3.5,2,0,"소고기, 꽃빵, 표고버섯, 홍피망, 청피망, 양파, 부추, 참깨, 고추맛 기름, 중...","소고기,꽃빵,표고버섯,홍피망,청피망,양파,부추,참깨",볶음
219,6,6645728787,15208828566,82429879997,프레시밀 고추잡채,"프레시밀 고추잡채, 1개, 580g",13200,0,13200,"(100g당 2,276원)",4.5,15,1,"또띠아, 돈 목살, 채소(양파, 피망, 표고버섯), 소스로 구성되어 있습니다.","또띠아,채소,양파,피망,표고버섯,소스",볶음
249,7,156649377,451066167,4117038986,프레시지 차돌박이 숙주 볶음,"프레시지 차돌박이 숙주 볶음, 401g, 1개",13900,21%,10900,"(100g당 2,718원)",4.5,14005,0,"소고기, 숙주나물, 양파, 소스 등으로 구성되어 있습니다.","소고기,숙주나물,양파,소스",볶음
320,7,6512230114,14388930125,81633200071,99프레시밀키트 태국식 삼겹살 볶음,"99프레시밀키트 태국식 삼겹살 볶음, 1개, 580g",13770,0,13770,"(100g당 2,374원)",4.5,34,1,"돼지고기, 숙주나물, 채소팩(양파, 대파, 청양고추), 마늘, 라임, 홍고추, 태국...","돼지고기,숙주나물,양파,대파,청양고추,마늘,라임,홍고추",볶음


# 쿠팡 상세카테고리 - 네이버 트렌드 데이터 연계 추천시스템 구현 

In [366]:
naver_df = pd.read_csv('2022-12-30edited_navertrend.csv', index_col = 0)
naver_df

,전체,밥/죽,찌개/국,면/파스타,구이,볶음/튀김,조림/찜,간식/디저트
0,밀키트,참한 전복,밀키트,밀키트,밀키트,밀키트,밀키트,떡볶이
1,떡볶이,리조또 밀키트,낙곱새,동남아식 밀키트 탄탄면,스테이크 밀키트,감바스 밀키트,캠핑요리추천,밀키트
2,낙곱새,프레시지 리조또,알탕,수제 비 밀키트,자미곱,감바스,캠핑음식추천,쌀떡볶이
3,캠핑 밀키트,옹기전 팥죽,캠핑 밀키트,파스타 밀키트,목살 밀키트,애슐리 밀키트,찜닭밀키트,국물떡볶이
4,알탕,빠에야 밀키트,해신탕,공항 칼국수,군고기 밀키트,닭갈비 밀키트,보일링크랩,사과떡볶이
5,쌀 떡볶이,기장 전복죽 밀키트,충무로 한우 곱창전골,애슐리 밀키트,초벌 한입 닭갈비,돈이 곱창 참숯 초벌 냄새없는 곱창 볶음 순대볶음 간단 캠핑 요리 밀키트 돼지 곱창...,갈비찜오리지널맛,떡볶이밀키트
6,해신탕,프레시지 트러플 크림 리조또,만두 전골,비빔국수 밀키트,석쇠 직화 술안주 캠핑 음식 술안주,춘천 닭갈비 택배 생생 닭갈비 3인 분 900 g 캠핑 요리,갈비찜밀키트650g3세트,즉석떡볶이
7,국물 떡볶이,김밥 밀키트,곱창전골 밀키트,라자냐 밀키트,직화구이 불닭갈비,오코노미야끼,우족찜,맛도령떡볶이
8,충무로 한우 곱창전골,죽 밀키트,곱창전골,소고기 버섯 칼국수,애슐리 밀키트,프레시지,바지락술찜,짜장떡볶이
9,사과 떡볶이,문어 솥밥 키트,마라탕 밀키트,뇨끼 밀키트,자미곱 대창,프레시지 감바스,아구찜,밀떡볶이


In [367]:
naver_df.replace('밀키트','', regex=True, inplace=True)

In [368]:
naver_df

,전체,밥/죽,찌개/국,면/파스타,구이,볶음/튀김,조림/찜,간식/디저트
0,,참한 전복,,,,,,떡볶이
1,떡볶이,리조또,낙곱새,동남아식 탄탄면,스테이크,감바스,캠핑요리추천,
2,낙곱새,프레시지 리조또,알탕,수제 비,자미곱,감바스,캠핑음식추천,쌀떡볶이
3,캠핑,옹기전 팥죽,캠핑,파스타,목살,애슐리,찜닭,국물떡볶이
4,알탕,빠에야,해신탕,공항 칼국수,군고기,닭갈비,보일링크랩,사과떡볶이
5,쌀 떡볶이,기장 전복죽,충무로 한우 곱창전골,애슐리,초벌 한입 닭갈비,돈이 곱창 참숯 초벌 냄새없는 곱창 볶음 순대볶음 간단 캠핑 요리 돼지 곱창 볶음...,갈비찜오리지널맛,떡볶이
6,해신탕,프레시지 트러플 크림 리조또,만두 전골,비빔국수,석쇠 직화 술안주 캠핑 음식 술안주,춘천 닭갈비 택배 생생 닭갈비 3인 분 900 g 캠핑 요리,갈비찜650g3세트,즉석떡볶이
7,국물 떡볶이,김밥,곱창전골,라자냐,직화구이 불닭갈비,오코노미야끼,우족찜,맛도령떡볶이
8,충무로 한우 곱창전골,죽,곱창전골,소고기 버섯 칼국수,애슐리,프레시지,바지락술찜,짜장떡볶이
9,사과 떡볶이,문어 솥밥 키트,마라탕,뇨끼,자미곱 대창,프레시지 감바스,아구찜,밀떡볶이


In [369]:
naver_df['전체_카테고리'] = naver_df['전체'].apply(lambda x : category_name(x))
naver_df['밥/죽_카테고리'] = naver_df['밥/죽'].apply(lambda x : category_name(x))
naver_df['찌개/국_카테고리'] = naver_df['찌개/국'].apply(lambda x : category_name(x))
naver_df['면/파스타_카테고리'] = naver_df['면/파스타'].apply(lambda x : category_name(x))
naver_df['구이_카테고리'] = naver_df['구이'].apply(lambda x : category_name(x))
naver_df['볶음/튀김_카테고리'] = naver_df['볶음/튀김'].apply(lambda x : category_name(x))
naver_df['조림/찜_카테고리'] = naver_df['조림/찜'].apply(lambda x : category_name(x))
naver_df['간식/디저트_카테고리'] = naver_df['간식/디저트'].apply(lambda x : category_name(x))

naver_df

,전체,밥/죽,찌개/국,면/파스타,구이,볶음/튀김,조림/찜,간식/디저트,전체_카테고리,밥/죽_카테고리,찌개/국_카테고리,면/파스타_카테고리,구이_카테고리,볶음/튀김_카테고리,조림/찜_카테고리,간식/디저트_카테고리
0,,참한 전복,,,,,,떡볶이,기타,기타,기타,기타,기타,기타,기타,떡볶이
1,떡볶이,리조또,낙곱새,동남아식 탄탄면,스테이크,감바스,캠핑요리추천,,떡볶이,리조또,기타,면,스테이크,감바스,기타,기타
2,낙곱새,프레시지 리조또,알탕,수제 비,자미곱,감바스,캠핑음식추천,쌀떡볶이,기타,리조또,알탕,기타,기타,감바스,기타,떡볶이
3,캠핑,옹기전 팥죽,캠핑,파스타,목살,애슐리,찜닭,국물떡볶이,기타,기타,기타,파스타,기타,기타,닭,떡볶이
4,알탕,빠에야,해신탕,공항 칼국수,군고기,닭갈비,보일링크랩,사과떡볶이,알탕,기타,해물탕,국수,기타,닭,기타,떡볶이
5,쌀 떡볶이,기장 전복죽,충무로 한우 곱창전골,애슐리,초벌 한입 닭갈비,돈이 곱창 참숯 초벌 냄새없는 곱창 볶음 순대볶음 간단 캠핑 요리 돼지 곱창 볶음...,갈비찜오리지널맛,떡볶이,떡볶이,기타,전골,기타,닭,순대,오리,떡볶이
6,해신탕,프레시지 트러플 크림 리조또,만두 전골,비빔국수,석쇠 직화 술안주 캠핑 음식 술안주,춘천 닭갈비 택배 생생 닭갈비 3인 분 900 g 캠핑 요리,갈비찜650g3세트,즉석떡볶이,해물탕,리조또,전골,국수,기타,닭,찜,떡볶이
7,국물 떡볶이,김밥,곱창전골,라자냐,직화구이 불닭갈비,오코노미야끼,우족찜,맛도령떡볶이,떡볶이,기타,전골,파스타,닭,기타,찜,떡볶이
8,충무로 한우 곱창전골,죽,곱창전골,소고기 버섯 칼국수,애슐리,프레시지,바지락술찜,짜장떡볶이,전골,기타,전골,국수,기타,기타,찜,면
9,사과 떡볶이,문어 솥밥 키트,마라탕,뇨끼,자미곱 대창,프레시지 감바스,아구찜,밀떡볶이,떡볶이,씨푸드,마라탕,기타,기타,감바스,찜,떡볶이


In [491]:
naver_df.loc[naver_df['볶음/튀김_카테고리'] == '볶음']

,전체,밥/죽,찌개/국,면/파스타,구이,볶음/튀김,조림/찜,간식/디저트,전체_카테고리,밥/죽_카테고리,찌개/국_카테고리,면/파스타_카테고리,구이_카테고리,볶음/튀김_카테고리,조림/찜_카테고리,간식/디저트_카테고리
19,크리스마스,기장끝집,우거지 감자탕 순살 해장국,레이식당,프레시지 블랙라벨,고추잡채,바지락술찜,즉석떡볶이460g,기타,기타,국,기타,기타,볶음,찜,떡볶이
22,캠핑 음식 추천,빠에야,만두 전골,피코크,춘천 황토집 숯불 닭갈비,곱창 볶음,성진한우우족찜,로제즉석캠핑떡볶이,기타,기타,전골,기타,닭,볶음,찜,떡볶이
23,홈파티,프레시지 트러플,속초 홍게 라면 캠핑 요리,콘 타이 정통 태국 소고기 쌀국수,솔리보,매운 쭈꾸미 오징어볶음 350 g,소제이소꼬리찜,밀떡로제짜장쌀떡볶이,기타,기타,면,국수,기타,볶음,찜,면
31,더원푸드 시래기 곱창전골,케일 쌈밥,샤브 샤브,프레시지,자미곱 곱창,오징어볶음,돼지갈비찜,옛날떡볶이캠핑,전골,쌈,기타,기타,기타,볶음,찜,떡볶이
34,동태 밥상,그랑씨엘,해물탕,조조 칼국수,폭립,고추잡채,파피요트,떡볶이밀떡세남자,기타,기타,해물탕,국수,기타,볶음,기타,떡볶이
36,밀푀유 나베,마구푸드,돼지국밥,돼고비,소곱창,낙지볶음,서민갑부김치찌개,식혜떡볶이유기농떡수제식혜부산어묵600g,전골,기타,국,기타,기타,볶음,찌개/짜글이,떡볶이
37,프레시지,먹물 빠에야,조개 구이 캠핑 조개찜,레이식당,마이야르,제육볶음,매운우족찜,마녀떡볶이3500원,기타,기타,구이,기타,기타,볶음,찜,떡볶이
70,라자냐,씨푸드 빠에야,술안주,파스타 맛집,쯔란 갈비,고추잡채 꽃 빵,포항아구찜,피자,파스타,씨푸드,기타,파스타,기타,볶음,찜,기타
93,조개 구이 캠핑 조개찜,빌라드 샬롯,부산 맛집 캠핑용,면 / 파스타,곱창 구이,잡채,쩐데이안동찜닭,신당동떡볶이택배,구이,기타,기타,면,구이,볶음,닭,떡볶이
95,갈비탕,수정과 키트,된장찌개,맹버칼,닭꼬치,볶음 / 튀김,복선당,신대장떡볶이,기타,기타,찌개/짜글이,기타,닭,볶음,기타,떡볶이


In [39]:
coupang

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리
0,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.5,15815,0,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기,새송이버섯,표고버섯,초간장소스,칠리소스,샤브용육수,생면,칼국수면,숙면,감자수...",전골
1,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,9%,23900,"(100g당 1,499원)",4.5,11387,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎...",전골
2,1,2202010823,3744622966,71729775756,미소프레쉬 6컵 푸짐한 대구탕,"미소프레쉬 6컵 푸짐한 대구탕, 1225g, 1개",14200,0,14200,"(100g당 1,159원)",4.5,12166,0,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물,채소,연두부,다시팩,소스",생선탕
3,1,1717552921,2923167957,70911802261,프레시지 더큰 햄가득 부대전골,"프레시지 더큰 햄가득 부대전골, 868g, 1개",14900,0,14900,"(100g당 1,717원)",5.0,36803,0,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지,오뗄팜S,프레스햄,소고기,치즈,라면사리,채소,김치,소스",전골
4,1,6310053892,13092768757,80353833768,마이셰프 EBS 최요비 산더미 소고기 콩불 2인분,"마이셰프 EBS 최요비 산더미 소고기 콩불 2인분, 810g, 1개",14900,0,14900,"(100g당 1,840원)",4.5,2942,0,"소고기, 콩나물, 대파, 양파, 깻잎, 소스로 구성되어 있습니다.","소고기,콩나물,대파,양파,깻잎,소스",한식
5,1,1637283332,2792368476,70782064297,곰곰 우삼겹 순두부 찌개,"곰곰 우삼겹 순두부 찌개, 710g, 1세트",8190,2%,7990,"(100g당 1,125원)",4.5,14386,0,"순두부, 애호박, 소고기, 팽이버섯, 대파, 소스","순두부,애호박,소고기,팽이버섯,대파,소스",찌개/짜글이
6,1,188130036,537230639,4402737129,프레시지 서울식 불고기 전골,"프레시지 서울식 불고기 전골, 424.5g, 1개",12900,15%,10900,"(100g당 2,568원)",4.5,8854,0,"소고기, 당면, 팽이버섯, 소스, 참기름 등으로 구성되어 있습니다.","소고기,당면,팽이버섯,소스,참기름",전골
7,1,1946869683,3305331651,71292240570,곰곰 애호박 된장찌개 밀키트,"곰곰 애호박 된장찌개 밀키트, 730g, 1개",8170,2%,7990,"(100g당 1,095원)",4.5,6585,0,"애호박, 양파, 대파, 청양고추, 두부, 팽이버섯","애호박,양파,대파,청양고추,두부,팽이버섯",찌개/짜글이
8,1,6764028903,15859811217,83069863236,프레시지 톡톡 터지는 알탕 (냉동),"프레시지 톡톡 터지는 알탕 (냉동), 1개, 765g",17900,8%,16380,"(100g당 2,141원)",4.5,589,0,"기타 수산물가공품(명태알, 대구곤이, 새우), 채소(무, 대파, 청양고추), 소스로...","명태알,대구곤이,새우,채소,무,대파,청양고추,소스",알탕
9,1,188130043,537230658,4402737156,프레시지 고깃집 된장찌개,"프레시지 고깃집 된장찌개, 535g, 1개",8900,15%,7500,"(100g당 1,402원)",4.5,12752,0,"소고기, 소스, 쥬키니호박, 팽이버섯, 대파, 청양고추, 두부로 구성되어 있습니다.","소고기,소스,쥬키니호박,팽이버섯,대파,청양고추,두부",찌개/짜글이


In [473]:
# 네이버트렌드 카테고리 : 상세카테고리
dict1 = {'밥/죽' : ['리조또', '한식'],
         '찌개/국' : ['전골', '생선탕', '찌개/짜글이', '알탕', '해물탕', '국', '마라탕'],
         '면/파스타' : ['면', '국수', '파스타'],
         '구이' : ['구이', '스테이크'],
         '볶음/튀김' : ['볶음', '감바스', '순대'],
         '조림/찜' : ['찜', '마파두부'],
         '간식/디저트' : ['샌드위치', '족발'],
         '전체' : ['씨푸드', '닭', '떡볶이', '무침', '쌈', '카레']}

In [474]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def coupang_naver_categories(category):
    
    if category in dict1['밥/죽'] : 
        naver_value = naver_df.loc[naver_df['밥/죽_카테고리'] == category]
        naver_value = naver_value['밥/죽'].values
        naver_value = naver_value.tolist()
    elif category in dict1['찌개/국'] : 
        naver_value = naver_df.loc[naver_df['찌개/국_카테고리'] == category]
        naver_value = naver_value['찌개/국'].values
        naver_value = naver_value.tolist()
    elif category in dict1['구이'] :
        naver_value = naver_df.loc[naver_df['구이_카테고리'] == category]
        naver_value = naver_value['구이'].values
        naver_value = naver_value.tolist()
    elif category in dict1['면/파스타'] : 
        naver_value = naver_df.loc[naver_df['면/파스타_카테고리'] == category]
        naver_value = naver_value['면/파스타'].values
        naver_value = naver_value.tolist()
    elif category in dict1['볶음/튀김'] : 
        naver_value = naver_df.loc[naver_df['볶음/튀김_카테고리'] == category]
        naver_value = naver_value['볶음/튀김'].values
        naver_value = naver_value.tolist()
    elif category in dict1['조림/찜'] : 
        naver_value = naver_df.loc[naver_df['조림/찜_카테고리'] == category]
        naver_value = naver_value['조림/찜'].values
        naver_value = naver_value.tolist()
    elif category in dict1['간식/디저트'] : 
        naver_value = naver_df.loc[naver_df['간식/디저트_카테고리'] == category]
        naver_value = naver_value['간식/디저트'].values
        naver_value = naver_value.tolist()
    elif category in dict1['전체'] : 
        naver_value = naver_df.loc[naver_df['전체_카테고리'] == category]
        naver_value = naver_value['전체'].values
        naver_value = naver_value.tolist()
    else:
        pass
    
    product_list = naver_value

    return category, product_list


def cv_commend_system(category, product_list):
    # 네이버 트렌드 상위 3개로부터 추출
    try:
        for num in range(3):
            docs = coupang.loc[coupang['상세카테고리'] == category].상품.values
            docs = np.append(docs,product_list[num])
            vect = CountVectorizer()
            countvect = vect.fit_transform(docs)
            countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))

            # 위의 Data Frame 형태의 유사도를 계산 
            cosine_matrix = cosine_similarity(countvect_df, countvect_df)

            productid = {}
            for i, c in enumerate(coupang.loc[coupang['상세카테고리'] == category].상품): productid[i] = c
            productid[len(productid)] = product_list[num]
            
            product = {}
            for i, c in productid.items(): product[c] = i

            idx = product[product_list[num]]
            sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
            sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)  
            print('현재 트렌드 상품 "', docs[-1], '"과 유사한 상품은',  sim_scores[0])        
        print(product)

    except:
        print('유사한 상품이 없어요')
        
    
def tfidf_commend_system(category, product_list):
    try:
        extracted_productid = []
        
        # 네이버 트렌드 상위 3개 키워드와 유사한 쿠팡 밀키트 추출
        for num in range(3):
            # TfidfVectorizer과 코사인 유사도를 이용한 추천시스템 
            docs = coupang.loc[coupang['상세카테고리'] == category].상품.values
            docs = np.append(docs,product_list[num])
            vect = TfidfVectorizer()
            tfidfvect = vect.fit_transform(docs)
            tfidfvect_df = pd.DataFrame(tfidfvect.toarray(), columns = sorted(vect.vocabulary_))
            
            # 위의 Data Frame 형태의 유사도를 계산 
            cosine_matrix = cosine_similarity(tfidfvect_df, tfidfvect_df)
            productid = {}
            for i, c in enumerate(coupang.loc[coupang['상세카테고리'] == category].상품): productid[i] = c
            productid[len(productid)] = product_list[num]
            
            product = {}
            for i, c in productid.items(): product[c] = i
            
            idx = product[product_list[num]]
            sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
            sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)  


            # 
            if len(extracted_productid) < 4:
                exproduct = productid[sim_scores[0][0]]
                if exproduct not in extracted_productid :
                    extracted_productid.append(exproduct)
                    print('현재 트렌드 -', docs[-1], '과 유사한 상품은', exproduct, sim_scores[0][1])
                else:
                    exproduct = productid[sim_scores[1][0]]
                    if exproduct not in extracted_productid :
                        extracted_productid.append(exproduct)
                        print('현재 트렌드 -', docs[-1], '과 유사한 상품은', exproduct, sim_scores[1][1])
                    else :
                        exproduct = productid[sim_scores[2][0]]
                        extracted_productid.append(exproduct)
                        print('현재 트렌드 -', docs[-1], '과 유사한 상품은', exproduct, sim_scores[2][1])
                    

    # 선택한 카테고리가 네이버 트렌드 데이터에 포함되어있지 않을 때
    except:
        try:
            coupang_sorted = coupang.loc[coupang['상세카테고리'] == category].sort_values('리뷰수', ascending = False)
            coupang_sorted.reset_index(inplace = True)
            if len(extracted_productid) == 2: 
                for num in range(5):
                    exproduct = coupang_sorted.loc[num].상품
                    if exproduct in extracted_productid :
                        pass
                    else :
                        extracted_productid.append(exproduct)
                        print(exproduct)
                        break
            elif len(extracted_productid) == 1:
                for num in range(5):
                    exproduct = coupang_sorted.loc[num].상품
                    if exproduct in extracted_productid :
                        pass
                    else :
                        extracted_productid.append(exproduct)
                        print(exproduct)
                        if len(extracted_productid) == 3:
                            break
            elif len(extracted_productid) == 0:
                for num in range(3):
                    exproduct = coupang_sorted.loc[num].상품
                    extracted_productid.append(exproduct)
                    print(exproduct)
                
        except:
            pass
    
    return extracted_productid
    


In [493]:
category, product_list = coupang_naver_categories('카레')
tfidf_commend = tfidf_commend_system(category, product_list)


마이셰프 티아시아 마크니 커리
마이셰프 티아시아 마살라 커리


In [447]:
tfidf_commend

['마이셰프 EBS 최고의 요리비결 냉채족발과 메밀면', '애슐리 마늘 족발과 막국수']

In [43]:
# 네이버트렌드 카테고리 : 상세카테고리
dict1 = {'찌개/국' : ['전골', '생선탕', '찌개/짜글이', '알탕', '해물탕', '국', '마라탕'],
         '면/파스타' : ['면', '국수', '파스타'],
         '구이' : ['구이', '스테이크'],
         '볶음/튀김' : ['볶음', '감바스', '순대'],
         '조림/찜' : ['찜', '마파두부', '족발'],
         '전체' : ['씨푸드', '닭', '떡볶이', '무침', '쌈', '샌드위치', '리조또', '카레']}

1. 네이버 트렌드 새로 데이터 추출해서, 띄어쓰기 사전 보충 O

2. 네이버 트렌드 데이터로 추천시스템 돌려보기

3. 조건식을 통해 유사도가 특정 수치 이하면, 그냥 패스시켜주고 그 카테고리인 애들 보여주는 정도? 여러 개중 리뷰수 기준?

4. 태현님 알레르기파일까지


1. 먼저 상세 카테고리를 입력받고 -> 상세카테고리에 따라서 적용할 네이버 트렌드 카테고리 받아주기

2. ex) 전골인 경우, 네이버트렌드 찌개/국 중 '전골' 포함되어있는 것 상위 n개 추출

3. 코사인 유사도를 이용하여, 쿠팡 데이터로부터 해당 키워드와 제일 유사한 상품들 추출

4. 만약 전골이 안에 없다면? 그냥 쿠팡에서 가져오기?

In [44]:
0.0 인애들은 그냥 다른걸로 대체?

Object `대체` not found.


In [ ]:
0.0 인애들은 그냥 다른걸로 대체